# Create same model PLIER DelayedPLIER

Marc Subirana-Granés (2024)

Create a basic PLIER model to compare between the results between PLIER and DelayedPLIER

# Load libraries/modules

In [2]:
`%>%` <- dplyr::`%>%`
library(PLIER)
library(dplyr)
library(reticulate)
library(here)

# PLIER utils
source(here::here('scripts/plier_util.R'))

#delayedPLIER functions from repo
path_script_funcs = '/home/msubirana/Documents/pivlab/DelayedPLIER/funcs.R'
source(path_script_funcs)

# Load PLIER pathway and cell type data
data(bloodCellMarkersIRISDMAP)
data(svmMarkers)
data(canonicalPathways)

Loading required package: RColorBrewer

Loading required package: gplots


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess


Loading required package: pheatmap

Loading required package: glmnet

Loading required package: Matrix

Loaded glmnet 4.1-8

Loading required package: knitr

Loading required package: rsvd

Loading required package: qvalue


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


here() starts at /home/msubirana/Documents/pivlab/plier_recount3

Loading required package: DelayedArray

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


Th

# Load data

In [3]:
# define output nb
output_nb_path = here('output/nbs/create_same_model_PLIER_DelayedPLIER')
dir.create(output_nb_path, showWarnings = FALSE)
expression_dataset_path <- here::here('output/gtex/GTEx_v8_gene_median_tpm.rds')

# Prepare data for all the models

In [5]:
# Load data
expression_dataset <- readRDS(expression_dataset_path)

# Remove gene ens id column and duplicate genes
expression_dataset <- subset(expression_dataset, select = -c(gene_ens_id))
expression_dataset <- expression_dataset[!duplicated(expression_dataset["gene_symbol"]),]

# Rename rows with gene symbols
rownames(expression_dataset) <- expression_dataset[,"gene_symbol"]

# Remove gene symbol column
expression_dataset <- subset(expression_dataset, select = -c(gene_symbol))

# Remove NA
expression_dataset = na.omit(expression_dataset)

# Convert to matrix
expression_matrix <- as.matrix(expression_dataset)

# Combine the pathway data from PLIER
all_paths <- PLIER::combinePaths(bloodCellMarkersIRISDMAP, svmMarkers, canonicalPathways)

# What genes are common to the pathway data and the expression matrix
cm_genes <- PLIER::commonRows(all_paths, expression_matrix)

# filter to common genes before row normalization to save on computation
expression_matrix_cm <- expression_matrix[cm_genes, ]

# Z-score normalization
expression_matrix_cm <- PLIER::rowNorm(expression_matrix_cm) 

# Remove NA
expression_matrix_cm=na.omit(expression_matrix_cm)

# What genes are common to the pathway data and the expression matrix
cm_genes <- PLIER::commonRows(all_paths, expression_matrix_cm)

# filter to common genes before row normalization to save on computation
expression_matrix_cm <- expression_matrix_cm[cm_genes, ]
all_paths_cm <- all_paths[cm_genes, ]

# PLIER preparation

In [29]:
# compute rsvd/svd
set.seed(123456)
ns=ncol(expression_matrix_cm)
message("Computing SVD")
if(ns>500){
  message("Using rsvd")
  set.seed(123456);svdres=rsvd(expression_matrix_cm, k=min(ns, max(200, ns/4)), q=3)
}else{
  svdres=svd(expression_matrix_cm)
}
message("Done")

# save z-scored expression data, the prior information matrix and svdres to be supplied to PLIER::PLIER and the number of PCs

output_file_preplier <- file.path(output_nb_path, 'preplier_comparison_dp_p.rds')


plier_data_list <- list("expression_matrix_cm" = expression_matrix_cm,
                        "all_paths_cm" = all_paths[cm_genes, ],
                        "svdres" = svdres)

saveRDS(plier_data_list, file = output_file_preplier)

Computing SVD

Using rsvd

Done



# PLIER 

In [28]:
# Assign arguments to variables 
output_file_plier <- file.path(output_nb_path, 'PLIER_comparison_dp_p.rds')
parameter_k <- 1
frac <- 0.7

# Load data
expression_matrix_cm=plier_data_list$expression_matrix_cm
all_paths_cm=plier_data_list$all_paths_cm
svdres=plier_data_list$svdres

# compute k
k=num.pc(svdres)*2
k <- min(k, floor(ncol(expression_matrix_cm)*0.9))
k = k * parameter_k
k = round(k, 0)
message("k is set to ", k)

# Run PLIER (with common genes)
plier_result=PLIER::PLIER(data=expression_matrix_cm, priorMat=all_paths_cm , svdres=svdres, k=k, frac=frac, scale=FALSE)

# Prepare output directory
output_file_path=dirname(output_file)
dir.create(dirname(output_file_path), showWarnings = FALSE, recursive = TRUE)

# Save results
saveRDS(plier_result, file = output_file_plier)

k is set to 1042

Removing 4 pathways with too few genes



[1] 55.53997
[1] "L2 is set to 55.5399692443312"
[1] "L1 is set to 27.7699846221656"


errorY (SVD based:best possible) = 0.6328

New L3 is 0.000804733010124613

New L3 is 0.000710174388842549

New L3 is 0.000710174388842549

New L3 is 0.000804733010124613

New L3 is 0.000804733010124613

New L3 is 0.000804733010124613

New L3 is 0.000911881965554516

New L3 is 0.000911881965554516

New L3 is 0.000911881965554516

New L3 is 0.000911881965554516

New L3 is 0.000911881965554516

New L3 is 0.000911881965554516

New L3 is 0.000911881965554516

New L3 is 0.000911881965554516

New L3 is 0.00103329763864764

New L3 is 0.000911881965554516

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

converged at  iteration 334 Bdiff is not decreasing

There are 216  LVs with AUC>0.70



# DelayedPLIER

In [6]:
output_file_preplier <- file.path(output_nb_path, 'preplier_comparison_dp_p.rds')
rds_preplier <- readRDS(output_file_preplier)
expression_matrix_cm <- rds_preplier$expression_matrix_cm
all_paths_cm <- rds_preplier$all_paths_cm
svdres <- rds_preplier$svdres

In [11]:
str(svdres)

List of 3
 $ d: num [1:4345] 5017 3058 2698 2573 2253 ...
 $ u: num [1:6683, 1:4345] -0.01033 -0.01441 -0.00257 -0.01231 -0.0053 ...
 $ v: num [1:17382, 1:4345] -0.00908 0.00523 -0.00754 -0.00811 0.00894 ...
 - attr(*, "class")= chr "rsvd"


In [12]:
head(all_paths_cm)

,IRIS_Bcell-Memory_IgG_IgA,IRIS_Bcell-Memory_IgM,IRIS_Bcell-naive,IRIS_CD4Tcell-N0,IRIS_CD4Tcell-Th1-restimulated12hour,IRIS_CD4Tcell-Th1-restimulated48hour,IRIS_CD4Tcell-Th2-restimulated12hour,IRIS_CD4Tcell-Th2-restimulated48hour,IRIS_CD8Tcell-N0,IRIS_DendriticCell-Control,⋯,KEGG_GNRH_SIGNALING_PATHWAY,KEGG_BASAL_TRANSCRIPTION_FACTORS,REACTOME_SYNTHESIS_OF_DNA,KEGG_HEMATOPOIETIC_CELL_LINEAGE,KEGG_T_CELL_RECEPTOR_SIGNALING_PATHWAY,PID_IL4_2PATHWAY,REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,PID_BCR_5PATHWAY,PID_TELOMERASEPATHWAY,PID_PI3KPLCTRKPATHWAY
GAS6,0,0,0,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0
MMP14,0,0,0,0,0,0,0,0,0,0,⋯,1,0,0,0,0,0,0,0,0,0
MARCKSL1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
SPARC,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
CTSD,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
EPAS1,0,0,0,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0


In [13]:
head(expression_matrix_cm)

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
GAS6,1.41132283,-0.4424523,0.3365641,0.5197589,-0.2412224,-0.36208833,0.64394272,-0.08622964,-0.3407591,0.04861881,⋯,-0.25188705,-0.4970079,0.02491961,-0.3627993,0.2116693,0.09815014,-0.5073644,0.28940268,-0.4370963,-0.09997518
MMP14,5.84234639,-0.7106564,0.7796052,0.4416584,-0.3322593,0.99025660,2.44930824,0.14829722,0.8254833,2.93587417,⋯,-0.48824475,-0.3727742,-0.22932447,-0.3190775,-0.2026377,0.29627013,-0.1134016,0.50304449,-0.6462979,1.35986578
MARCKSL1,-0.22726833,-0.5812718,-0.5171569,0.0883272,-0.3056592,-0.04935212,0.02888631,-0.44171275,-0.2897575,-0.41397367,⋯,-0.41722514,-0.2556679,0.04412756,-0.3047447,-0.5572414,3.34690689,-0.5867993,-0.55414238,-0.5909348,-0.49703849
SPARC,1.63786217,-0.5300667,0.1609265,1.2996054,-0.2172089,0.94117200,0.54119827,-0.67367746,0.1383761,-0.39037262,⋯,0.01043193,-0.1437419,0.08567921,-0.6826976,-0.5610439,0.03725509,-0.7826080,0.57917797,-0.7380056,1.53935231
CTSD,0.63014415,0.6176855,-0.4076144,0.3904307,0.2229312,0.09488407,-0.40715297,0.24900206,0.2787644,0.74642478,⋯,0.14587220,4.2712970,-0.30010096,-0.2032004,-0.7140047,-0.73569192,0.6428335,0.03535945,0.1989368,0.28222509
EPAS1,-0.07935625,-0.5188999,0.3340869,0.9338027,-0.3057833,0.40919083,0.09018520,-0.47919152,0.8847249,-0.45022819,⋯,-0.19275569,-0.3906284,1.04794570,-0.4974098,-0.4316753,-0.55871985,-0.5876088,1.50079985,-0.6065335,0.68432390


In [22]:
head(svdres$d)

[1] 5017.043 3058.120 2697.594 2573.171 2253.001 2103.693

In [14]:
output_file_delayedPLIER_hdf5 <- file.path(output_nb_path, 'counts.hdf5')  
output_file_delayedPLIER_dim <- file.path(output_nb_path, 'dimnames.RDS') 

writeHDF5Array(expression_matrix_cm, filepath = output_file_delayedPLIER_hdf5, name = "count")
saveRDS(list(row.names = rownames(expression_matrix_cm) , col.names = colnames(expression_matrix_cm)), file = output_file_delayedPLIER_dim)

<6683 x 17382> HDF5Matrix object of type "double":
              [,1]       [,2]       [,3] ...    [,17381]    [,17382]
   [1,]  1.4113228 -0.4424523  0.3365641   . -0.43709630 -0.09997518
   [2,]  5.8423464 -0.7106564  0.7796052   . -0.64629791  1.35986578
   [3,] -0.2272683 -0.5812718 -0.5171569   . -0.59093477 -0.49703849
   [4,]  1.6378622 -0.5300667  0.1609265   . -0.73800558  1.53935231
   [5,]  0.6301441  0.6176855 -0.4076144   .  0.19893678  0.28222509
    ...          .          .          .   .           .           .
[6679,] -0.2462670 -0.7701677 -0.5296683   .  -0.7600938  -0.2738677
[6680,] -0.1980982 -0.2400479 -0.1990823   .  -0.2451261  -0.2109961
[6681,]  2.1421737 -0.7290364  1.5123376   .  -0.9276022   0.7712268
[6682,] -0.6538122 -0.9657483 -0.6224588   .  -1.0976921  -0.7310973
[6683,] -0.7506101 -0.9781575 -0.6702703   .  -0.8675906  -0.6017146

In [27]:
output_file_svdres_hdf5 <- file.path(output_nb_path, 'svd.hdf5')  
h5createFile(output_file_svdres_hdf5)
h5createGroup(output_file_svdres_hdf5, "svd")
h5write(svdres$d, output_file_svdres_hdf5, "svd/d")
h5write(svdres$u, output_file_svdres_hdf5, "svd/u")
h5write(svdres$v, output_file_svdres_hdf5, "svd/v")

file '/home/msubirana/Documents/pivlab/plier_recount3/output/nbs/create_same_model_PLIER_DelayedPLIER/svd.hdf5' already exists.

Can not create group. Object with name 'svd' already exists.



ERROR: Error in UseMethod("h5writeDataset"): no applicable method for 'h5writeDataset' applied to an object of class "c('HDF5Matrix', 'HDF5Array', 'DelayedMatrix', 'DelayedArray', 'DelayedUnaryIsoOp', 'DelayedUnaryOp', 'DelayedOp', 'Array', 'RectangularData')"


## Check for correct PLIER (delayedPLIER repo)function

In [35]:
str(PLIER)

function (data, priorMat, svdres = NULL, k = NULL, L1 = NULL, L2 = NULL, 
    L3 = NULL, frac = 0.7, max.iter = 350, trace = F, scale = T, Chat = NULL, 
    maxPath = 10, doCrossval = T, penalty.factor = rep(1, ncol(priorMat)), 
    glm_alpha = 0.9, minGenes = 10, tol = 1e-06, seed = 123456, allGenes = F, 
    rseed = NULL, pathwaySelection = c("complete", "fast"), output_path = "output/")  


In [28]:
setAutoRealizationBackend("HDF5Array") #supportedRealizationBackends(), getRealizationBackend()

# Load each component of the SVD results
d <- HDF5Array(filepath = output_file_svdres_hdf5, name = "svd/d")
u <- HDF5Array(filepath = output_file_svdres_hdf5, name = "svd/u")
v <- HDF5Array(filepath = output_file_svdres_hdf5, name = "svd/v")

# Reassemble the SVD results into a list
svdres <- list(d = as.numeric(d), u = DelayedArray(u), v = DelayedArray(v))
class(svdres) <- "rsvd"

sce <- DelayedArray(seed = HDF5ArraySeed(filepath = output_file_delayedPLIER_hdf5, name = "count"))
dimnamaes <- readRDS(output_file_delayedPLIER_dim)
rownames(sce) <- dimnamaes$row.names
colnames(sce) <- dimnamaes$col.names

sce[is.na(sce)] <- 0
expression_matrix_dp <- sce[which(DelayedMatrixStats::rowSds(sce) >0),]

# Assign arguments to variables 
output_file_delayedPlier <- file.path(output_nb_path, 'delayedPlier_comparison_dp_p.rds')
parameter_k <- 1
frac <- 0.7

# compute k
k=num.pc(svdres)*2
k <- min(k, floor(ncol(expression_matrix_cm)*0.9))
k = k * parameter_k
k = round(k, 0)
message("k is set to ", k)

# Run PLIER (with common genes)
delayedPlier_result=PLIER(data=expression_matrix_dp, priorMat=all_paths_cm , svdres=svdres, k=k, frac=frac, scale=FALSE)

# Prepare output directory
output_file_path=dirname(output_file_delayedPlier)
dir.create(dirname(output_file_delayedPlier), showWarnings = FALSE, recursive = TRUE)

# Save results
saveRDS(delayedPlier_result, file = output_file_delayedPlier)

k is set to 1042

Removing 4 pathways with too few genes



[1] 55.53997
[1] "L2 is set to 55.5399692443312"
[1] "L1 is set to 27.7699846221656"


ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'a' in selecting a method for function 'solve': when the right operand is not a DelayedArray object (or derivative),
  its length (1085764) cannot be greater than the first dimension of the
  left operand (1042)


In [112]:
# Check the dimensions of your matrices
dim(expression_matrix_dp)
dim(all_paths_cm)

# Check the dimensions of svdres
str(svdres)

[1]  6683 17382

[1] 6683  628

List of 3
 $ d: num [1:4345] 5017 3058 2698 2573 2253 ...
 $ u: num [1:6683, 1:4345] -0.01033 -0.01441 -0.00257 -0.01231 -0.0053 ...
 $ v: num [1:17382, 1:4345] -0.00908 0.00523 -0.00754 -0.00811 0.00894 ...
 - attr(*, "class")= chr "rsvd"


-0.009084249,0.0048984083,0.003729253,-5.068619e-03,-0.001676492,0.0012167764,0.0004521597,-1.731784e-03,0.0028415248,-0.0108770568,⋯,0.010316521,-0.0018216236,0.009547465,0.0007294641,-0.0107351040,-0.003858802,-0.001301917,-0.0065631243,0.015982440,0.0059481690
0.005227597,-0.0006597834,0.007296205,5.113093e-03,0.004106840,0.0058876053,-0.0220267426,1.199124e-02,0.0042095739,-0.0094073135,⋯,0.005766039,0.0031255668,-0.017016444,0.0020608788,0.0178844464,0.010818435,0.008424042,-0.0046739341,-0.006138034,-0.0042348002
-0.007538556,0.0024068316,0.006820188,-6.764531e-03,0.001602638,0.0026827639,-0.0056995087,-3.174828e-03,-0.0118093426,-0.0031382752,⋯,0.008278355,0.0001931016,-0.005094602,0.0028290347,0.0009537677,0.002132480,0.012981176,-0.0209944783,-0.001496447,-0.0032482439
-0.008108760,0.0015281041,0.004592110,-7.836804e-03,0.002823339,0.0022566112,0.0031078898,-1.156762e-04,0.0001972283,-0.0040141988,⋯,-0.012390217,-0.0031648826,0.009541483,0.0027030098,-0.0007984402,-0.012082119,0.010887267,-0.0022341602,-0.004085697,-0.0060873099
0.008938876,0.0006916050,0.004663470,8.402255e-05,0.001085982,0.0026497992,0.0033395725,2.563578e-03,-0.0026726362,-0.0002996245,⋯,-0.006185586,-0.0048198469,0.003343766,-0.0021676098,0.0022386804,0.003489230,0.007948025,-0.0008856395,0.002323073,-0.0007892155
0.002806485,0.0052999336,0.003398989,-3.305127e-03,0.000603851,0.0004749406,0.0070639287,-2.017282e-05,-0.0012545485,-0.0106848200,⋯,0.012219089,0.0194636303,0.011813677,0.0098170893,-0.0210587766,-0.002653762,0.006859786,-0.0036161516,-0.004182263,0.0163174848


In [ ]:
head(delayedPlier_result$Z)

In [ ]:
head(delayedPlier_result$B)

# run delayedPLIER step by step

In [30]:
data=expression_matrix_dp
priorMat=all_paths_cm
svdres=NULL
k=k
L1=NULL
L2=NULL
L3=NULL
frac=frac
max.iter=350
trace=F
scale=FALSE
Chat=NULL
maxPath=10
doCrossval=T
penalty.factor=rep(1,ncol(priorMat))
glm_alpha=0.9
minGenes=10
tol=1e-6
seed=123456
allGenes=F
rseed=NULL
pathwaySelection=c("complete", "fast")
output_path = "output/"

In [32]:
pathwaySelection=match.arg(pathwaySelection, c("complete", "fast"))

if(scale){
Y=rowNorm(data)
} else{
Y=data
}

if(nrow(priorMat)!=nrow(data) || !all(rownames(priorMat)==rownames(data))){
if(!allGenes){
  cm=commonRows(data, priorMat)
  message(paste("Selecting common genes:", length(cm)))
  priorMat=priorMat[cm,]
  Y=Y[cm,]
} else{
  extra.genes=setdiff(rownames(data), rownames(priorMat))
  eMat=matrix(0, nrow=length(extra.genes), ncol=ncol(priorMat))
  rownames(eMat)=extra.genes
  priorMat=rbind(priorMat, eMat)
  priorMat=priorMat[rownames(data),]
}

}
numGenes=colSums(priorMat)

heldOutGenes=list()
iibad=which(numGenes<minGenes)
priorMat[, iibad]=0
message(paste("Removing", length(iibad), "pathways with too few genes"))
if(doCrossval){


priorMatCV=priorMat
if(!is.null(seed))
  set.seed(seed)
for(j in 1:ncol(priorMatCV)){
  
  iipos=which(priorMatCV[,j]>0)
  iiposs=sample(iipos, length(iipos)/5)
  priorMatCV[iiposs,j]=0
  heldOutGenes[[colnames(priorMat)[j]]]=rownames(priorMat)[iiposs]
  
}#for j
C = priorMatCV
}else{
C=priorMat
}#else

nc=ncol(priorMat)
ng=nrow(data)
ns=ncol(data)

Bdiff=-1
BdiffTrace=double()
BdiffCount=0
if(is.null(Chat)){
Cp=crossprod(C)
Chat=pinv.ridge(crossprod(C), 5)%*%(t(C))
}

Yseq = Y^2
YsqSum=sum(DelayedMatrixStats::rowSums2(Yseq))

Removing 4 pathways with too few genes



In [33]:
#compute svd and use that as the starting point
if(!is.null(svdres) && nrow(svdres$v)!=ncol(Y)){
message("SVD V has the wrong number of columns")
svdres=NULL
}

if(is.null(svdres)){
message("Computing SVD")
if(ns>500){
  message("Using rsvd")
  set.seed(123456);
  #svdres=rsvd(Y, k=min(ns, max(200, ns/4)), q=3)
  svdres = BiocSingular::runRandomSVD(Y, k = min(ns, max(200, ns/4)), center = F, scale = F)
} else{
  svdres=BiocSingular::runRandomSVD(Y, k = min(ng, ns))
}
message("Done")
}

if(is.null(k)){
k=num.pc(svdres)*2
k <- min(k, floor(ncol(Y)*0.9))
message("k is set to ", k)
}

if(is.null(L2)){
show(svdres$d[k])
L2=svdres$d[k]
print(paste0("L2 is set to ",L2))
}

if(is.null(L1)){
L1=L2/2
print(paste0("L1 is set to ",L1))
}

Computing SVD

Using rsvd

Done



[1] 55.53993
[1] "L2 is set to 55.5399316449024"
[1] "L1 is set to 27.7699658224512"


In [ ]:
B=t(svdres$v[1:ncol(Y), 1:k]%*%diag(svdres$d[1:k]))
Z=(Y%*%t(B))%*%solve(tcrossprod(B)+L1*diag(k))
Z[Z<0]=0
if(!is.null(rseed)){
message("using random start")
set.seed(rseed)
B=t(apply(B, 1, sample))
Z=apply(Z,2,sample)
}

B <- DelayedArray(B)
U=matrix(0,nrow=ncol(C), ncol=k)

round2=function(x){signif(x,4)}
message(paste0("errorY (SVD based:best possible) = ", round2(mean((Y-Z%*%B)^2))))


iter.full.start=iter.full=20

curfrac=0
nposlast=Inf
npos=-Inf
if(!is.null(L3)){
L3.given=T
}else{
L3.given=F
}#else